In [ ]:
savedmode_name="/model_seed42_score0.991.pth"

In [ ]:
# %pip install --force-reinstall /kaggle/input/unidecode/Unidecode-1.3.8-py3-none-any.whl
# %cp /kaggle/input/utility/* .

In [ ]:
# TODO:change path
from cfg import *
from util import *
from model1 import NLPModel
import gc

# Config.data_path='/kaggle/input/pii-detection-removal-from-educational-data/'
# Config.modelsavepath='/kaggle/input/deberbase0213'
tokenizer_path=Config.modelsavepath
modelpath=Config.modelsavepath+savedmode_name
modelpath

In [ ]:
Config.modelsavepath

In [ ]:
import json
from itertools import chain
from transformers import AutoModelForTokenClassification

In [ ]:
data = json.load(open(Config.data_path+"/train.json"))

In [ ]:
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
})
ds

In [ ]:
tokenizer_path

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
idlabel= json.load(open(Config.modelsavepath+"//idlabel.json"))
id2label=idlabel['id2label']
label2id=idlabel['label2id']
id2label

In [ ]:
def inference_preprocesss(example,tokenizer,label2id):
    # rebuild text from tokens

    example['token_map']=[]
    example['berttokenids']=[]
    example['berttokenmask']=[]
    example['berttokentoken_type_ids']=[]
    example['offset_mapping']=[]

    text = []
    token_map = []
    
    idx = 0
    
    for t, ws in zip(example["tokens"], example["trailing_whitespace"]):
        
        text.append(t)
        token_map.extend([idx]*len(t))
        if ws:
            text.append(" ")
            token_map.append(-1)
            
        idx += 1
        
        
        # actual tokenization
    tokenizeds = tokenizer("".join(text),
                        stride=Config.stride,
                        max_length=Config.max_length,
                        truncation=True ,
                        return_overflowing_tokens=True,
                        return_offsets_mapping=True,
                        )


    example['berttokenids']=(tokenizeds['input_ids'])
    example['berttokenmask']=(tokenizeds['attention_mask'])
    example['berttokentoken_type_ids']=(tokenizeds['token_type_ids'])
    example['offset_mapping']=(tokenizeds['offset_mapping'])
    example['token_map']=(token_map)
        
    return example

In [ ]:
preprocesssed_ds=ds.map(inference_preprocesss, fn_kwargs={'tokenizer':tokenizer,'label2id':label2id},num_proc=10,desc="prepocessing data")

In [ ]:
# rebuid dataset 
tmp_pd=expanddataset(preprocesssed_ds,if_train=False)
print(tmp_pd['berttokenids'].str.len().agg(['mean','max','std','min']))
print(len(tmp_pd))

In [ ]:
full_ds=Dataset.from_pandas(tmp_pd)
full_ds

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=NLPModel(id2label,label2id,Config.modelsavepath,training=False).to(device)
model.load_state_dict(torch.load(modelpath))

In [ ]:
data_collator = Collate(tokenizer=tokenizer,if_train=False)
val_dataloader=DataLoader(full_ds,batch_size=1,pin_memory=True,collate_fn=data_collator)
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def ret_before_pading_logit(batch_logits,batch_org_len):
    """
        按顺序返回pading前的logit列表
    """
    logit_list=[]
    batch_len=max(batch_org_len)
    for i,l in enumerate(batch_org_len):
        loggit=batch_logits[i*batch_len:i*batch_len+l]
        logit_list.append(loggit)
    return logit_list

In [ ]:
model.eval()

logits=[]
for step,dataset in enumerate(tqdm(val_dataloader)):

    with torch.no_grad():
        ids = dataset["ids"].to(device,non_blocking=True)
        mask = dataset["mask"].to(device,non_blocking=True)
        tokentype = dataset["type_ids"].to(device,non_blocking=True)
        logit,loss = model(ids,mask,tokentype)
        logits+=(ret_before_pading_logit(logit,dataset['token_org_length']))
        del ids,mask,tokentype


In [ ]:
triplet_map = {}
document, token, label, token_str = [], [], [], []

for p,token_map, offsets,tokens, doc in zip(logits,full_ds["token_map"], full_ds['offset_mapping'],full_ds["tokens"], full_ds["document"]):

    for token_pred, (start_idx, end_idx) in zip(p, offsets):
        
        if start_idx + end_idx == 0: continue
        if token_map[start_idx] == -1:
            start_idx += 1

         # ignore "\n\n"
        while start_idx < len(token_map) and  tokens[token_map[start_idx]].isspace():
            start_idx += 1

        if start_idx >= len(token_map): break

        token_id = token_map[start_idx]

        # ignore "O" predictions and whitespace preds
        if token_id != -1:
            triplet_key = (doc,token_id,tokens[token_id])

            if triplet_key not in triplet_map:
                triplet_map[triplet_key]=[token_pred]
            else:
                triplet_map[triplet_key].append(token_pred)

In [ ]:
for key in triplet_map:
    averged_logit=np.mean(triplet_map[key],axis=0)
    pred_softmax= np.exp(averged_logit) / np.sum(np.exp(averged_logit))
    if pred_softmax[12]> Config.threshold:
        true_predict='O'
    else:
        true_predict=id2label[str(pred_softmax[:12].argmax())]
    triplet_map[key]=true_predict

In [ ]:
df=pd.DataFrame(triplet_map.items(), columns=['triplet_map', 'label'])
df=df.loc[df['label']!='O'].reset_index(drop=True)

In [ ]:
df['document'],df['token'], df['token_str'] =  zip(*df.triplet_map)

In [ ]:
df["row_id"] = list(range(len(df)))
(df.head(100))

In [ ]:
df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)